In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
from math import sqrt
from sklearn import model_selection
from scipy.stats import spearmanr

In [3]:
csvdata = pd.read_csv("rand_forest.csv")

In [13]:
csvdata

,cof,ASA_m^2/g,Density,LS,B,O,C,H,Si,N,S,Ni,Zn,Cu,Co,F,P,Cl,V,Br
0,19542N2,1581.73,0.662891,27.65854,0.000000,0.000000,0.571429,0.285714,0.0,0.085714,0.057143,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20191N2,2590.76,0.332648,33.01596,0.000000,0.000000,0.565217,0.347826,0.0,0.043478,0.043478,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20202N2,2620.64,0.354451,30.65329,0.000000,0.000000,0.608696,0.347826,0.0,0.043478,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19102N2,1706.93,0.578502,23.82296,0.000000,0.000000,0.631579,0.263158,0.0,0.105263,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16121N2,1515.80,0.766045,16.31523,0.000000,0.118644,0.461017,0.298305,0.0,0.108475,0.013559,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,13050N2,1731.87,0.614558,22.25156,0.000000,0.111111,0.555556,0.277778,0.0,0.055556,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562,16272N2,2795.55,0.355595,39.21249,0.000000,0.000000,0.578947,0.385965,0.0,0.035088,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
563,18101N2,2322.70,0.419450,28.47879,0.000000,0.055556,0.555556,0.333333,0.0,0.055556,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
564,17110N2,3337.00,0.359030,25.71565,0.000000,0.029412,0.470588,0.470588,0.0,0.029412,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
db_cofs = csvdata[['cof']]
db_traindata = csvdata[['ASA_m^2/g','Density','LS','B','O','C','H',
                        'Si','N','S','Ni','Zn','Cu','Co','F','P','Cl','V','Br']]

In [11]:
db_cofs = np.squeeze(db_cofs.values)

In [89]:
db_traindata = db_traindata.values

In [15]:
mask_0 = np.load('masks0.npy')

In [25]:
ones_0 = np.where(mask_0[:,0] == 1)[0]
zero_0 = np.where(mask_0[:,0] == 0)[0]

In [48]:
train_input_0 = np.array([db_traindata.values[i] for i in ones_0])
test_input_0 = np.array([db_traindata.values[i] for i in zero_0])

In [50]:
target_data = pd.read_csv('targets.csv')

In [51]:
target_data

,name,h2_77K_1bar,h2_77K_5bar,h2_77K_25bar,h2_77K_50bar,h2_77K_75bar,h2_77K_100bar,h2_198K_1bar,h2_198K_5bar,h2_198K_25bar,...,co2_0.001bar,co2_30bar,ch4_1.0bar,ch4_5.8bar,ch4_20bar,ch4_35bar,ch4_50bar,ch4_65bar,n2_0.001bar,n2_30bar
0,20610N2,8.608329,18.943764,33.019479,42.110343,48.077294,52.526693,0.220603,1.091591,4.971186,...,0.000810,26.302520,0.585023,3.042712,8.728171,12.857054,15.901036,18.234089,0.000312,5.919703
1,21052N2,12.492448,24.194547,40.296970,48.517777,53.710548,56.759958,0.275615,1.299559,5.850303,...,0.006273,31.889790,0.884184,4.093504,10.228093,14.376330,17.335400,19.667728,0.000333,7.177639
2,20473N2,7.351015,18.898034,39.891623,56.159212,69.075600,79.111762,0.323447,1.542330,7.367576,...,0.000925,26.675046,0.604803,3.221392,9.432452,14.509765,18.741625,22.436572,0.000306,7.842990
3,15161N2,11.103453,19.127537,29.268909,34.150457,36.880407,38.670559,0.182272,0.874597,3.948490,...,0.000886,15.112881,0.620940,3.012800,7.472873,10.186392,11.969428,13.352719,0.000133,3.857558
4,17061N2,11.473854,15.031012,17.343098,18.156936,18.552978,18.818870,0.158416,0.736588,2.972030,...,0.001605,8.408278,1.288961,4.313877,6.421067,7.150720,7.564887,7.780053,0.000178,3.354187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,15040N2,12.116210,18.747770,24.913016,27.053079,28.107507,29.039463,0.183287,0.884612,3.775029,...,0.001136,13.838464,1.042827,4.912427,9.347823,11.005380,11.922382,12.523654,0.000156,4.331875
568,20640N2,9.978542,19.437045,29.850485,34.551719,37.729733,39.279700,0.190556,0.933708,4.136128,...,0.001246,21.826607,0.719193,3.695986,9.188567,12.298091,14.246543,15.530680,0.000266,5.548890
569,16056N2,9.601632,16.119540,25.716533,31.354143,34.988888,37.406117,0.198080,0.947045,4.090201,...,0.001211,16.176539,0.814866,3.729888,8.310867,10.992854,12.977538,14.389489,0.000229,4.546194
570,21013N2,9.128204,18.733539,32.621997,42.024845,47.935489,52.415050,0.218663,1.109665,4.897949,...,0.001184,25.494649,0.590074,3.094270,8.556674,12.549903,15.515264,17.841339,0.000224,5.950226


In [59]:
list1 = np.squeeze(target_data[['name']].values)

In [58]:
list2 = np.squeeze(db_cofs.values)

In [65]:
valid_list = list(set(list1).intersection(list2))

In [64]:
target_data = target_data.values

In [71]:
rem = []
for i,cof in enumerate(target_data):
    if cof[0] not in valid_list:
        rem.append(i)

In [80]:
target_data_cleaned = np.delete(target_data, rem, 0)

In [83]:
target_data_cleaned[:,0].shape

(566,)

In [85]:
resorted_cofs = []
for name in list2:
    resorted_cofs.append(np.where(target_data_cleaned[:,0] == name))

In [88]:
resorted_cofs = np.squeeze(resorted_cofs)

In [92]:
db_traindata_resorted = db_traindata[resorted_cofs]

In [93]:
target_data_cleaned

array([['20610N2', 8.608328660304, 18.943764395251, ..., 18.234088704696,
        0.000311527250125, 5.9197031123253],
       ['21052N2', 12.492448150163, 24.194546921414, ...,
        19.667728454469, 0.0003326980257342, 7.1776394358039],
       ['20473N2', 7.3510147145437, 18.898033666196, ..., 22.43657183939,
        0.0003064909313025, 7.8429900143194],
       ...,
       ['16056N2', 9.6016317564762, 16.119540361344, ...,
        14.389489134272, 0.00022854498135, 4.5461939145161],
       ['21013N2', 9.1282042520172, 18.733538631658, ...,
        17.841339486487, 0.0002235725646713, 5.9502256427729],
       ['19292N2', 12.682301795869, 18.935153557068, ...,
        11.801085470329, 0.0002667161337814, 4.5566382892874]],
      dtype=object)

In [98]:
target_data2 = pd.read_csv('properties.csv')

In [99]:
target_data2

,name,h2_77K_1bar,h2_77K_5bar,h2_77K_25bar,h2_77K_50bar,h2_77K_75bar,h2_77K_100bar,h2_198K_1bar,h2_198K_5bar,h2_198K_25bar,...,n2_henry,n2_henry_unit,xe_henry,xe_henry_unit,kr_henry,kr_henry_unit,h2o_henry,h2o_henry_unit,h2s_henry,h2s_henry_unit
0,20610N2,8.608329,18.943764,33.019479,42.110343,48.077294,52.526693,0.220603,1.091591,4.971186,...,0.000002,mol/kg/Pa,0.000022,mol/kg/Pa,0.000006,mol/kg/Pa,0.000003,mol/kg/Pa,0.000021,mol/kg/Pa
1,21052N2,12.492448,24.194547,40.296970,48.517777,53.710548,56.759958,0.275615,1.299559,5.850303,...,0.000004,mol/kg/Pa,0.000068,mol/kg/Pa,0.000013,mol/kg/Pa,0.000041,mol/kg/Pa,0.000164,mol/kg/Pa
2,20473N2,7.351015,18.898034,39.891623,56.159212,69.075600,79.111762,0.323447,1.542330,7.367576,...,0.000003,mol/kg/Pa,0.000021,mol/kg/Pa,0.000007,mol/kg/Pa,0.000026,mol/kg/Pa,0.000019,mol/kg/Pa
3,15161N2,11.103453,19.127537,29.268909,34.150457,36.880407,38.670559,0.182272,0.874597,3.948490,...,0.000002,mol/kg/Pa,0.000033,mol/kg/Pa,0.000007,mol/kg/Pa,0.000008,mol/kg/Pa,0.000029,mol/kg/Pa
4,17061N2,11.473854,15.031012,17.343098,18.156936,18.552978,18.818870,0.158416,0.736588,2.972030,...,0.000002,mol/kg/Pa,0.000144,mol/kg/Pa,0.000017,mol/kg/Pa,0.000009,mol/kg/Pa,0.000122,mol/kg/Pa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,15040N2,12.116210,18.747770,24.913016,27.053079,28.107507,29.039463,0.183287,0.884612,3.775029,...,0.000002,mol/kg/Pa,0.000072,mol/kg/Pa,0.000012,mol/kg/Pa,0.000004,mol/kg/Pa,0.000059,mol/kg/Pa
568,20640N2,9.978542,19.437045,29.850485,34.551719,37.729733,39.279700,0.190556,0.933708,4.136128,...,0.000003,mol/kg/Pa,0.000034,mol/kg/Pa,0.000008,mol/kg/Pa,0.000004,mol/kg/Pa,0.000030,mol/kg/Pa
569,16056N2,9.601632,16.119540,25.716533,31.354143,34.988888,37.406117,0.198080,0.947045,4.090201,...,0.000002,mol/kg/Pa,0.000049,mol/kg/Pa,0.000009,mol/kg/Pa,0.000007,mol/kg/Pa,0.000045,mol/kg/Pa
570,21013N2,9.128204,18.733539,32.621997,42.024845,47.935489,52.415050,0.218663,1.109665,4.897949,...,0.000003,mol/kg/Pa,0.000024,mol/kg/Pa,0.000007,mol/kg/Pa,0.000214,mol/kg/Pa,0.000031,mol/kg/Pa


In [100]:
target_data2[['h2o_henry', 'h2s_henry', 'xe_henry', 'kr_henry', 'co2_0.001bar', 'o2_5bar', 'o2_140bar', 'co2_30bar', 'n2_0.001bar', 'n2_30bar', 'h2_77K_5bar', 'h2_77K_100bar', 
            'h2_298K_5bar', 'h2_298K_100bar', 'ch4_65bar', 'ch4_5.8bar']]

,h2o_henry,h2s_henry,xe_henry,kr_henry,co2_0.001bar,o2_5bar,o2_140bar,co2_30bar,n2_0.001bar,n2_30bar,h2_77K_5bar,h2_77K_100bar,h2_298K_5bar,h2_298K_100bar,ch4_65bar,ch4_5.8bar
0,0.000003,0.000021,0.000022,0.000006,0.000810,1.287679,20.609347,26.302520,0.000312,5.919703,18.943764,52.526693,0.515931,8.921144,18.234089,3.042712
1,0.000041,0.000164,0.000068,0.000013,0.006273,1.839179,22.116277,31.889790,0.000333,7.177639,24.194547,56.759958,0.538124,8.986940,19.667728,4.093504
2,0.000026,0.000019,0.000021,0.000007,0.000925,1.507559,29.767765,26.675046,0.000306,7.842990,18.898034,79.111762,0.825823,15.011729,22.436572,3.221392
3,0.000008,0.000029,0.000033,0.000007,0.000886,1.120439,14.566810,15.112881,0.000133,3.857558,19.127537,38.670559,0.339126,5.762190,13.352719,3.012800
4,0.000009,0.000122,0.000144,0.000017,0.001605,1.499412,8.529768,8.408278,0.000178,3.354187,15.031012,18.818870,0.214550,3.230384,7.780053,4.313877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,0.000004,0.000059,0.000072,0.000012,0.001136,1.519165,12.942924,13.838464,0.000156,4.331875,18.747770,29.039463,0.301524,4.742890,12.523654,4.912427
568,0.000004,0.000030,0.000034,0.000008,0.001246,1.309162,16.155864,21.826607,0.000266,5.548890,19.437045,39.279700,0.365491,6.221505,15.530680,3.695986
569,0.000007,0.000045,0.000049,0.000009,0.001211,1.382149,15.727506,16.176539,0.000229,4.546194,16.119540,37.406117,0.381156,6.367652,14.389489,3.729888
570,0.000214,0.000031,0.000024,0.000007,0.001184,1.289830,20.223637,25.494649,0.000224,5.950226,18.733539,52.415050,0.520836,8.865178,17.841339,3.094270


In [13]:
obj = pd.read_pickle(r'splits/split_run_{}.pkl'.format(1))

In [15]:
obj['masks']['test']



array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 1, 0]], dtype=int64)